#### Obtener una metica sobre cualquier tema
##### - Obtener los datos para el calculo de la metrica
##### - La formula de calculo de la metrica
##### - Descripcion de lo que hace la metrica y el objetivo que persigue
##### - Poner un ejemplo de uso de esa metrica

In [26]:
#Importar las librerias
import requests
import os
from dotenv import load_dotenv
from pandas import DataFrame
import time

#Carga del archivo .env con los datos para uso de la api
load_dotenv()

#Guardar los datos necesarios para la conexion en constantes
API_KEY= os.getenv("API_KEY")
REGION= "americas"
INVOCADOR= "neoded"

In [27]:
#Funcion para obtener el indicador de jugador unico puuid
def obtener_puuid(invocador_id, region, api_key):
    url = f"https://{region}.api.riotgames.com//riot/account/v1/accounts/by-riot-id/{invocador_id}/LAN"
    headers= {"X-Riot-Token": api_key}
    response = requests.get(url, headers= headers)
    time.sleep(0.05)#Evitar exceder el numero de peticiones por segundo

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        raise Exception(f"Error al obtener PUUID: {response.status_code} - {response.text}")

puuid = obtener_puuid(INVOCADOR, REGION, API_KEY)

In [30]:
#Obtener una cantidad de partidas recientes
def obtener_partidas_recientes(puuid, region, api_key, cantidad):
    url= f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?count={cantidad}"
    headers = {"X-Riot-Token": api_key}
    respuesta= requests.get(url, headers= headers)
    time.sleep(0.05)#Evitar exceder el numero de peticiones por segundo

    if respuesta.status_code == 200:
        return respuesta.json()
    else:
        raise Exception(f"Error al obtener Match IDs: {respuesta.status_code} - {respuesta.text}")

partidas_recientes= obtener_partidas_recientes(puuid, REGION, API_KEY, 20)
print(partidas_recientes)

['LA1_1631339440', 'LA1_1631321927', 'LA1_1631317465', 'LA1_1631117376', 'LA1_1631113256', 'LA1_1631106620', 'LA1_1630992363', 'LA1_1630984977', 'LA1_1630977330', 'LA1_1630962876', 'LA1_1630955789', 'LA1_1630864539', 'LA1_1630863054', 'LA1_1630855691', 'LA1_1630639964', 'LA1_1630635219', 'LA1_1630622015', 'LA1_1630609739', 'LA1_1630594843', 'LA1_1630575798']


In [37]:
#Funcion para obtener los detalles de una partida
def obtener_detalles_partida(partida_id, region, api_key):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{partida_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    time.sleep(0.05)#Evitar exceder el numero de peticiones por segundo

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error al obtener detalles del match: {response.status_code} - {response.text}")
    
#Obtener los detalles de las partidas
detalles_partidas= [obtener_detalles_partida(partida_id, REGION, API_KEY) for partida_id in partidas_recientes]

In [33]:
#Funcion para obtener los detalles de la partida del jugador seleccionado
def datos_jugador_partida(partidas, puuid_jugador):
    detalles= []
    for partida in partidas:
        #Obtener los participantes
        participantes= partida.get("info").get("participants")
        #Iterar a traves de los participantes para encontrar el identificador del jugador seleccionado
        for participante in participantes:
            if participante.get("puuid") == puuid_jugador:
                detalles.append(participante)

    #Devolver los detalles de partida
    return detalles

#Obtener los detalles de partidas del jugador
detalles_jugador= datos_jugador_partida(detalles_partidas, puuid)

In [34]:
#Crear un dataframe con los datos
df= DataFrame(detalles_jugador)

In [35]:
df

,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,PlayerScore8,PlayerScore9,allInPings,assistMePings,assists,baronKills,basicPings,challenges,champExperience,champLevel,championId,championName,championSkinId,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,...,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,0,34,0,0,"{'12AssistStreakCount': 1, 'HealFromMapSources...",15852,16,901,Smolder,0,0,0,0,257,257,257,13740,0,7,0,1,0,True,0,0,False,False,...,8,0,1,13,4,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,974,,False,200,,14,941,0,45104,25788,0,24758,0,2967,192,23,159,170,5,0,537,537,505,0,1,0,0,0,0,0,0,0,True
1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,4,18,0,0,"{'12AssistStreakCount': 1, 'HealFromMapSources...",26631,18,50,Swain,33,0,7,5,1805,5419,1805,57516,0,5,3,0,0,True,1,0,False,False,...,42,0,8,12,6,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,100,MIDDLE,88,2922,7,316877,71043,0,71840,0,37931,442,275,634,284,5,0,24165,5481,2884,0,4,6,0,0,37,3,3,19,True
2,8,8,0,0,867.834167,0.234127,4.150001,3195.783447,3090.821289,752.375,21.329411,0,0,0,3,0,0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",12000,13,14,Sion,8,0,0,0,0,0,0,25484,0,5,0,0,0,True,0,0,False,False,...,8,8,4,2201,3,2202,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,200,,45,1499,0,32257,25566,0,24726,0,6019,0,0,170,650,1,0,6300,6300,5107,0,0,0,0,0,0,0,0,0,False
3,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,1,21,0,0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",21210,18,202,Jhin,3,0,0,2,2715,2715,2715,11750,0,8,0,4,0,True,0,0,True,False,...,50,0,7,3,5,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,100,,47,1263,0,90309,56602,0,20152,0,2345,0,40,135,241,1,2,768,768,307,1,3,0,0,0,0,0,0,0,True
4,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,0,18,0,0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",12474,14,31,Chogath,38,0,0,0,766,766,766,14136,0,8,0,1,0,True,0,0,False,False,...,12,0,28,32,2,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,200,,89,827,0,37608,22974,0,20969,0,1539,0,17,231,172,1,0,10297,4498,1406,0,2,0,0,0,0,0,0,0,True
5,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,0,32,0,0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",15751,16,126,Jayce,35,0,0,0,3040,3040,3040,15770,0,6,0,3,0,True,0,0,True,False,...,60,0,6,14,3,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,100,,20,869,0,55125,39576,0,14544,0,1343,0,23,73,133,1,1,1872,1872,664,0,4,0,0,0,0,0,0,0,True
6,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0,0,0,42,0,0,"{'12AssistStreakCount': 1, 'HealFromMapSources...",33914,18,235,Senna,9,0,0,1,3719,3719,3719,25656,0,11,0,3,0,True,0,0,False,False,...,11,0,8,3,9,4,bsm5kM2aucMiFP4WZ0w67xA4c7TuC8to2gKsKggq5gnORq0,973,,False,200,,68,1629,0,180651,83661,6655,37249,0,19685,5295,62,285,297,5,0,2085,2085,3784,4,4,1,0,0,0,0,0,0,True
7,0,0,0,0,0.000000,0.000000,0.000

In [36]:
#Obtener el porcentaje de victorias en los ultimos 5 juegos
porcentaje= df.win.mean()
print("El porcentaje de victorias es igual a {}%".format(porcentaje*100))

El porcentaje de victorias es igual a 70.0%
